#this notebook will be mainly used for the capstone project

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Week3 - Clustering Toronto's neighborhoods

In [68]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Go to the page in wikipedia

In [69]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)
url=r.content
soup = BeautifulSoup(url,'html.parser')

### choosing the table from the page

In [70]:
My_table=soup.find("table",{"class":"wikitable sortable"})

table_rows = My_table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [71]:
df=pd.DataFrame(l, columns=["PostalCode", "Borough","Neighborhood"])

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [72]:
df['Neighborhood'] = df['Neighborhood'].str.replace('\n', '')

df1 = df[df.Borough != 'Not assigned']

In [73]:
print('Table head:')
print(df1.head())
print('Table shape:',df1.shape)

Table head:
  PostalCode           Borough      Neighborhood
0       None              None              None
3        M3A        North York         Parkwoods
4        M4A        North York  Victoria Village
5        M5A  Downtown Toronto      Harbourfront
6        M6A        North York  Lawrence Heights
Table shape: (211, 3)


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page,
#### you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.
#### These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [74]:
df2=df1.groupby('PostalCode')['Neighborhood'].apply(','.join).to_frame().reset_index('PostalCode')
df2=df2.rename(columns={"Neighborhood": "Neighborhoods"})
df4=df1.drop(columns=['Neighborhood'])
df3= pd.merge(df4, df2, how='inner', left_on = 'PostalCode', right_on = 'PostalCode')

df5=df3.drop_duplicates()

print(df5.head(10))
print('Table shape:',df5.shape)

   PostalCode           Borough                    Neighborhoods
0         M3A        North York                        Parkwoods
1         M4A        North York                 Victoria Village
2         M5A  Downtown Toronto                     Harbourfront
3         M6A        North York  Lawrence Heights,Lawrence Manor
5         M7A      Queen's Park                     Not assigned
6         M9A  Downtown Toronto                     Queen's Park
7         M1B       Scarborough                    Rouge,Malvern
9         M3B        North York                  Don Mills North
10        M4B         East York   Woodbine Gardens,Parkview Hill
12        M5B  Downtown Toronto          Ryerson,Garden District
Table shape: (103, 3)


In [75]:
df5['Neighborhoods'] = np.where((df5.Neighborhoods == 'Not assigned'),df5.Borough,df5.Neighborhoods)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [76]:
print(df5.head(10))
print('Table shape:',df5.shape)

   PostalCode           Borough                    Neighborhoods
0         M3A        North York                        Parkwoods
1         M4A        North York                 Victoria Village
2         M5A  Downtown Toronto                     Harbourfront
3         M6A        North York  Lawrence Heights,Lawrence Manor
5         M7A      Queen's Park                     Queen's Park
6         M9A  Downtown Toronto                     Queen's Park
7         M1B       Scarborough                    Rouge,Malvern
9         M3B        North York                  Don Mills North
10        M4B         East York   Woodbine Gardens,Parkview Hill
12        M5B  Downtown Toronto          Ryerson,Garden District
Table shape: (103, 3)
